# Filtrado de imágenes

## Contenidos

- Filtrado con máscaras en frecuencia
- Filtrado con convolución 2D


In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from scipy import fftpack
from ipywidgets import interact, IntSlider

def color2bw(img):
    return np.dot(img, [0.299, 0.587, 0.114])

img_example = color2bw(plt.imread("../images/InsInformatica.jpg"))  


> Una multiplicación en el espacio de frecuencia equivale a una convolución en el espacio original

¿Qué ocurre con la imagen original cuando énmascaramos una parte del espectro?

### Filtro pasa-bajos: Eliminando las frecuencias altas

Forma 1: Multiplicando el espectro de magnitud por una ventana rectangular

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 3.5), tight_layout=True);

cy, cx = img_example.shape[0]/2, img_example.shape[1]/2
x = np.arange(0, img_example.shape[1]); 
y = np.arange(0, img_example.shape[0]);
X, Y = np.meshgrid(x, y)
S_img = fftpack.fft2(img_example)

def update(sigma):
    for ax_ in ax:
        ax_.cla()
        ax_.axis('off')
    mask = np.zeros_like(S_img, dtype=np.float32)
    mask[int(cy-sigma):int(cy+sigma), 
         int(cx-sigma):int(cx+sigma)] = 1
    im = ax[1].imshow(fftpack.fftshift(np.log(1+np.abs(S_img)))*mask, cmap=plt.cm.Spectral_r)
    reconstructed_img = np.real(fftpack.ifft2(fftpack.ifftshift(fftpack.fftshift(S_img)*mask)))
    im = ax[0].imshow(reconstructed_img, cmap=plt.cm.Greys_r)
    
interact(update, 
         sigma=IntSlider(min=1, max=200.0, value=200, description="Width"));

Forma 2: Multiplicando el espectro de magnitud por una ventana Gaussiana

In [ ]:
from ipywidgets import FloatSlider
fig, ax = plt.subplots(1, 2, figsize=(8, 3.5), tight_layout=True);

def update(sigma):
    for ax_ in ax:
        ax_.cla()
        ax_.axis('off')
    mask = 1e-8 + np.exp(-(((X-cx)/sigma)**2 + ((Y-cy)/sigma)**2))
    im = ax[1].imshow(fftpack.fftshift(np.log(1+np.abs(S_img)))*mask, cmap=plt.cm.Spectral_r)
    reconstructed_img = np.real(fftpack.ifft2(fftpack.ifftshift(fftpack.fftshift(S_img)*mask)))
    im = ax[0].imshow(reconstructed_img, cmap=plt.cm.Greys_r)
    
interact(update, 
         sigma=FloatSlider(min=1, max=200.0, value=200, description="$\sigma$"));

### Filtro pasa-altos: Eliminando las frecuencias bajas

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 3.5), tight_layout=True);

def update(sigma=1):
    for ax_ in ax:
        ax_.cla()
        ax_.axis('off')
    mask = 1.0  - np.exp(-(((X-cx)/sigma)**2 + ((Y-cy)/sigma)**2)) 
    im = ax[1].imshow(fftpack.fftshift(np.log(1+np.abs(S_img)))*mask, cmap=plt.cm.Spectral_r)
    reconstructed_img = np.real(fftpack.ifft2(fftpack.ifftshift(fftpack.fftshift(S_img)*mask)))
    im = ax[0].imshow(reconstructed_img, cmap=plt.cm.Greys_r)
    
interact(update, sigma=FloatSlider(min=1, max=100.0, value=1, description="$\sigma$"));

### Filtro pasa-banda y rechaza-banda

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 3.5), tight_layout=True);

def update(sigma1=1, sigma2=1):
    for ax_ in ax:
        ax_.cla()
        ax_.axis('off')
    mask1 = np.exp(-(((X-cx)/sigma1)**2 + ((Y-cy)/sigma1)**2)) 
    mask2 = np.exp(-(((X-cx)/sigma2)**2 + ((Y-cy)/sigma2)**2)) 
    mask = mask1 - mask2
    im = ax[1].imshow(fftpack.fftshift(np.log(1+np.abs(S_img)))*mask, cmap=plt.cm.Spectral_r)
    reconstructed_img = np.real(fftpack.ifft2(fftpack.ifftshift(fftpack.fftshift(S_img)*mask)))
    im = ax[0].imshow(reconstructed_img, cmap=plt.cm.Greys_r)
    
interact(update, 
         sigma1=FloatSlider(min=1, max=200.0, value=200, description="$\sigma_1$"),
         sigma2=FloatSlider(min=1, max=200.0, value=1, description="$\sigma_2$"));

## Convolucion 2D

También podemos filtrar una imagen en su espacio original usando la convolución bidimensional

El elemento que se convoluciona con la imagen se denomina filtro o kernel de convolución


<img src="../images/filter2D_convolution.gif" width="600">


> ¿Qué hacen estos los siguientes kernels/filtros?

$$
\begin{pmatrix}
1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 \\
\end{pmatrix} \frac{1}{25} 
\qquad
\begin{pmatrix}
0.018 & 0.082 & 0.1353 & 0.082 & 0.018 \\
0.082 & 0.3678 & 0.6065 & 0.3678 & 0.082 \\
0.1353 & 0.6065 & 1 & 0.6065 & 0.1353 \\
0.082 & 0.3678 & 0.6065 & 0.3678 & 0.082 \\
0.018 & 0.082 & 0.1353 & 0.082 & 0.018 \\
\end{pmatrix} \frac{1}{\sqrt{2\pi}}
$$




Scipy ofrece dos funciones para hacer convolución

- convolve2d: Convolución tradicional (más rápido cuando la imagen y el filtro son pequeños)
- fftconvolve: Convolución multiplicando en frecuencia (más rápido cuando la imagen y el filtro son grandes)

In [ ]:
from scipy.signal import fftconvolve, convolve2d

fig, ax = plt.subplots(1, 2, figsize=(8, 3.5), tight_layout=True);

def update(size=1):
    for ax_ in ax:
        ax_.cla()
        ax_.axis('off')
    kernel = np.ones(shape=(size, size))/size**2
    img_filtered1 = fftconvolve(img_example, kernel, mode='same');
    img_filtered2 = convolve2d(img_example, kernel, mode='same');
    im = ax[0].imshow(img_filtered1, cmap=plt.cm.Greys_r)
    im = ax[1].imshow(img_filtered2, cmap=plt.cm.Greys_r)
    
interact(update, size=IntSlider(min=1, max=100.0, value=1, description="Size"));

### Detección de borde con filtro Sobel

Los siguientes filtros se conocen como sobel horizontal y vertical

<img src="../images/filtro_gradient.gif" width="600">

In [ ]:
fig = plt.figure(figsize=(7, 7), tight_layout=True)
sobelx = fftconvolve(img_example, [[-1, 0, 1],[-2, 0, 2], [-1, 0, 1]], mode='full')
ax =  plt.subplot2grid((2, 2), (0, 0))
ax.matshow(sobelx, cmap=plt.cm.Greys_r); 
ax.axis('off')
sobely = fftconvolve(img_example, [[-1, -2, -1],[0, 0, 0], [1, 2, 1]], mode='full')
ax = plt.subplot2grid((2, 2), (0, 1))
ax.matshow(sobely, cmap=plt.cm.Greys_r);
ax.axis('off')
ax = plt.subplot2grid((2, 2), (1, 0), colspan=2)
ax.matshow(np.sqrt(sobely**2 + sobelx**2)[3:-3,3:-3], cmap=plt.cm.Greys_r); 
ax.axis('off');
